# 다중 분류
    이진 분류가 0 또는 1 , 불량 또는 정상 등 클래스가 2개 라면, 다중 분류는 클래스가 3개 이상이다.
    이진 분류와 차이는 평가지표에만 차이가 있다.

Q. 개인의 신용 관련 정보 데이터다. 개인의 신용 등급(Credit_Score)를 예측하시오
  - 평가 기준은 macro-f1 으로 평가
  - label(target)은 신용 등급(Credit_Score) : 1,2,3
  - 제출 방식은 test 데이터로 예측한 class 1개 컬러만 csv로 제출한다.( 컬럼명 : pred, 파일 : result.csv)

In [54]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (3).csv
Saving test.csv to test (3).csv


In [55]:
import pandas as pd

train = pd.read_csv("train (3).csv")
test = pd.read_csv("test (3).csv")
train.shape,test.shape

((10000, 21), (10000, 20))

In [56]:
train.head(5)

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Monthly_Balance,Credit_Score,Credit_Mix,...,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month
0,56.0,16.0,11.0,35.598217,120.0,Yes,229.093478,252.385965,1,Bad,...,15.0,36597.56,8.0,10.0,29.0,5.0,3143.796667,22.49,2963.18,122.900223
1,49.0,23.0,12.0,25.553106,120.0,Yes,104.613906,219.105944,1,Bad,...,28.0,32057.30,9.0,8.0,16.0,7.0,2606.441667,1.40,1327.26,164.859426
2,34.0,20.0,6.0,40.039954,174.0,Yes,338.626965,251.265589,1,Bad,...,46.0,75868.80,6.0,10.0,32.0,7.0,6074.400000,3.60,1432.71,297.547446
3,21.0,13.0,8.0,25.711678,143.0,NM,116.816864,259.927960,2,Standard,...,46.0,17092.69,7.0,3.0,19.0,7.0,1695.390833,16.40,1417.06,62.794260
4,19.0,13.0,6.0,39.140463,138.0,Yes,87.262887,626.212330,1,Bad,...,45.0,81471.96,6.0,6.0,25.0,5.0,6763.330000,27.09,2679.69,202.857783


In [58]:
train.isnull().sum()
test.isnull().sum()

,0
Delay_from_due_date,0
Num_of_Delayed_Payment,0
Num_Credit_Inquiries,0
Credit_Utilization_Ratio,0
Credit_History_Age,0
Payment_of_Min_Amount,0
Amount_invested_monthly,0
Monthly_Balance,0
Credit_Mix,0
Payment_Behaviour,0


In [62]:
train['Credit_Score'].value_counts()

,count
Credit_Score,
2,5237
1,2978
3,1785


In [63]:
target = train.pop('Credit_Score')
cols = train.select_dtypes(include='O').columns
cols

Index(['Payment_of_Min_Amount', 'Credit_Mix', 'Payment_Behaviour'], dtype='object')

In [68]:
# 인코딩없이 자료형을 'category'로 변환 후 LightGBM

for col in cols:
  train[col] = train[col].astype('category')
  test[col] = test[col].astype('category')

In [70]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Delay_from_due_date       10000 non-null  float64 
 1   Num_of_Delayed_Payment    10000 non-null  float64 
 2   Num_Credit_Inquiries      10000 non-null  float64 
 3   Credit_Utilization_Ratio  10000 non-null  float64 
 4   Credit_History_Age        10000 non-null  float64 
 5   Payment_of_Min_Amount     10000 non-null  category
 6   Amount_invested_monthly   10000 non-null  float64 
 7   Monthly_Balance           10000 non-null  float64 
 8   Credit_Mix                10000 non-null  category
 9   Payment_Behaviour         10000 non-null  category
 10  Age                       10000 non-null  float64 
 11  Annual_Income             10000 non-null  float64 
 12  Num_Bank_Accounts         10000 non-null  float64 
 13  Num_Credit_Card           10000 non-null  float

In [73]:
# 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(train,target,test_size=0.2,random_state=0)

X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((8000, 20), (2000, 20), (8000,), (2000,))

In [74]:
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier()
lgbmc.fit(X_tr,y_tr)
pred = lgbmc.predict(X_val)
pred

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000561 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2529
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 20
[LightGBM] [Info] Start training from score -1.208148
[LightGBM] [Info] Start training from score -0.643881
[LightGBM] [Info] Start training from score -1.737271


array([2, 2, 2, ..., 3, 2, 2])

In [77]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def check(y_val,pred):
  accuracy = accuracy_score(y_val,pred)
  print('accuracy_score: ',accuracy)
  f1 = f1_score(y_val,pred,average='macro')
  print('f1_score: ',f1)

check(y_val,pred)

accuracy_score:  0.6985
f1_score:  0.6777379561595467


In [79]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.fit_transform(X_val)

# randomForest는 category 자료형을 인식하지 못한다.


ValueError: could not convert string to float: 'Yes'

In [81]:
pred = lgbmc.predict(test)
pred

array([2, 1, 1, ..., 1, 1, 2])

In [82]:
sumbit = pd.DataFrame({'pred':pred})
sumbit.to_csv("result.csv",index=False)

pd.read_csv("result.csv")

,pred
0,2
1,1
2,1
3,2
4,1
...,...
9995,2
9996,2
9997,1
9998,1
